Embeds input using Stanford GloVe (common crawl data) and saves it.

In [ ]:
import pandas as pd
from tqdm import tqdm
from random import randint
import numpy as np
import json
import pickle
import re
from google.colab import drive
drive.mount('/drive')
import os

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [ ]:
file_name = 'reviews_Amazon_Instant_Video_5.json'

In [ ]:
# Extracting the GloVe embedder (https://nlp.stanford.edu/projects/glove/)

with open('/drive/My Drive/Colab Notebooks/6.7900 project/glove_twitter_25d.pickle', 'rb') as handle:
    embedder = pickle.load(handle)
d = len(embedder[','])
print(drive)

<module 'google.colab.drive' from '/usr/local/lib/python3.8/dist-packages/google/colab/drive.py'>


In [ ]:
# Extract data

reviews = []
scores = []
n = 0
with open("/drive/My Drive/Colab Notebooks/6.7900 project/" + file_name, "r") as file:
    for line in file:
        loaded = json.loads(line)
        reviews.append(loaded["reviewText"])
        scores.append(loaded["overall"])
        n += 1

In [ ]:
n

37126

In [ ]:
# Create input and label

label = [[0,0,0,0,0] for _ in scores]
inp = reviews
for idx in range(n):
    curr_score = int(scores[idx]) - 1
    label[idx][curr_score] = 1

In [ ]:
# Randomize

for idx in range(1, n):
    rand = randint(0, idx)
    label[idx], label[rand] = label[rand], label[idx]
    inp[idx], inp[rand] = inp[rand], inp[idx]

In [ ]:
# Embed

new_input = []
new_label = []
max_words = 200  # maximum number of words
m = 0
for i in range(n):
    split_sentence = re.findall(r"\w+|[^\w\s]", inp[i], re.UNICODE)
    if len(split_sentence) >= max_words:
        split_sentence = split_sentence[len(split_sentence)-max_words:]
    
    new_input.append([])
    for _ in range(max_words - len(split_sentence)):
        new_input[-1].append(np.zeros(d))
    new_input[-1].extend([np.array(embedder.get(word, [0] * d)) for word in split_sentence])

    new_label.append(label[i])
    m += 1
    if i <= 20:
        print(split_sentence)

new_input = np.array(new_input)
new_label = np.array(new_label)

print(m)

['Joe', 'and', 'the', 'bots', 'are', 'put', 'through', 'a', 'grueling', 'ordeal', 'w', '/', 'LOST', 'CONTINENT', '.', 'They', 'hang', 'on', 'for', 'dear', 'life', 'during', 'the', 'interminably', '-', 'long', '"', 'mountain', 'climbing', '"', 'sequence', 'that', 'consumes', 'nearly', 'half', 'the', 'movie', "'", 's', 'running', 'time', 'w', '/', 'climbing', '.', '.', '.', 'Climbing', '.', '.', '.', 'CLIMBING', '!', 'A', 'dinosaur', '(', 'aka', ':', 'a', 'tiny', 'lizard', 'filmed', 'from', 'below', ')', 'is', 'glimpsed', ',', 'but', 'even', 'it', 'leaves', 'out', 'of', 'boredom', '.', 'Finally', ',', 'a', 'stop', '-', 'motion', 'brontosaurus', 'lumbers', 'forth', ',', 'only', 'to', 'be', 'filled', 'w', '/', 'lead', '!', 'So', 'little', 'action', 'takes', 'place', ',', 'that', 'Joel', 'nearly', 'lapses', 'into', 'a', 'coma', '!', 'Luckily', ',', 'through', 'the', 'wonders', 'of', 'MST3K', 'technology', ',', 'this', 'film', 'is', 'rendered', '(', 'nearly', ')', 'harmless', '.', 'Let', 'us

In [ ]:
print(new_input.shape)
print(new_label.shape)
print(new_input[0])

(37126, 200, 25)
(37126, 5)
[[ 0.       0.       0.      ...  0.       0.       0.     ]
 [ 0.       0.       0.      ...  0.       0.       0.     ]
 [ 0.       0.       0.      ...  0.       0.       0.     ]
 ...
 [ 0.69586 -1.1469  -0.41797 ... -0.67877  0.7061   0.40833]
 [ 0.69586 -1.1469  -0.41797 ... -0.67877  0.7061   0.40833]
 [ 0.69586 -1.1469  -0.41797 ... -0.67877  0.7061   0.40833]]


In [ ]:
# Save data

if os.path.exists('/drive/My Drive/Colab Notebooks/6.7900 project/reviews_amazon_instant_video_twitterglove.npy'):
    os.remove('/drive/My Drive/Colab Notebooks/6.7900 project/reviews_amazon_instant_video_twitterglove.npy')
    print('path deleted')

with open('/drive/My Drive/Colab Notebooks/6.7900 project/reviews_amazon_instant_video_twitterglove.npy', 'wb') as f:
    np.save(f, new_input)
    np.save(f, new_label)

path deleted
